<a href="https://colab.research.google.com/github/CryptoRobotFr/easy_backtest/blob/main/dca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-binance

In [5]:
# Importation des librairies
import pandas as pd
from binance.client import Client
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Variables pour les données
client = Client()
pair_symbol = "BTCUSDT"
time_interval = Client.KLINE_INTERVAL_1WEEK
start_date = "01 january 2017"

# Récupération des données
klinesT = client.get_historical_klines(pair_symbol, time_interval, start_date)

# Créer un tableau grâce aux données
df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])

# Supprime les colonnes inutiles
df.drop(columns = df.columns.difference(['timestamp','open','high','low','close','volume']), inplace=True)

# Convertit les colonnes en numéric
for col in df.columns:
    df[col] = pd.to_numeric(df[col])

# Convertit les dates dans un format lisible
df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')
del df['timestamp']

# Affiche le tableau
df

,open,high,low,close,volume
timestamp,,,,,
2017-08-14,4261.48,4485.39,3850.00,4086.29,2843.431426
2017-08-21,4069.13,4453.91,3400.00,4310.01,4599.396629
2017-08-28,4310.01,4939.19,4124.54,4509.08,4753.843376
2017-09-04,4505.00,4788.59,3603.00,4130.37,6382.787745
2017-09-11,4153.62,4394.59,2817.00,3699.99,8106.705127
...,...,...,...,...,...
2022-03-21,41262.11,46999.00,40467.94,46827.76,315436.150440
2022-03-28,46827.76,48189.84,44200.00,46407.35,312053.159640
2022-04-04,46407.36,47200.00,41868.00,42158.85,268118.291110


In [6]:
df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)

# -- Indicators, you can edit every value --
df['LAST_ATH'] = df['close'].cummax()

print("Indicators loaded 100%")

# -- Uncomment the line below if you want to check your dataset with indicators --
df

Indicators loaded 100%


,open,high,low,close,volume,LAST_ATH
timestamp,,,,,,
2017-08-14,4261.48,4485.39,3850.00,4086.29,2843.431426,4086.29
2017-08-21,4069.13,4453.91,3400.00,4310.01,4599.396629,4310.01
2017-08-28,4310.01,4939.19,4124.54,4509.08,4753.843376,4509.08
2017-09-04,4505.00,4788.59,3603.00,4130.37,6382.787745,4509.08
2017-09-11,4153.62,4394.59,2817.00,3699.99,8106.705127,4509.08
...,...,...,...,...,...,...
2022-03-21,41262.11,46999.00,40467.94,46827.76,315436.150440,65519.10
2022-03-28,46827.76,48189.84,44200.00,46407.35,312053.159640,65519.10
2022-04-04,46407.36,47200.00,41868.00,42158.85,268118.291110,65519.10


In [9]:
# Achète toutes les semaines le même montant
dfTest = df[:]

weeklyAmount = 30
takerFee = 0.0007
buyAmount = weeklyAmount
mediumBuy = 0
totalInvest = 0
btcWallet = 0

for index, row in dfTest.iterrows():
    totalInvest += buyAmount
    buyBTC = buyAmount / row['close']
    btcWallet += buyBTC - takerFee * buyBTC
    mediumBuy += 1

resultInDollar = btcWallet * dfTest.iloc[-1]['close']
perfInPct = (resultInDollar - totalInvest)/totalInvest
buyAndHoldPerf = (dfTest.iloc[-1]['close'] - dfTest.iloc[0]['close'])/dfTest.iloc[0]['close']
print('Buy',mediumBuy,'time',weeklyAmount,'$')   
print('Total invest', totalInvest, '$')
print('Final wallet', round(btcWallet,3), 'BTC')
print('Final wallet equivalent', round(resultInDollar,2), '$')
print('Performance',round(perfInPct*100,2), '%')
print('Buy and Hold performance', round(buyAndHoldPerf*100,2), '%')

Buy 245 time 30 $
Total invest 7350 $
Final wallet 0.782 BTC
Final wallet equivalent 31016.62 $
Performance 321.99 %
Buy and Hold performance 871.02 %


In [11]:
# DCA optimisé achète en fonction du dernier ATH

dfTest = df[:]

weeklyAmount = 30
takerFee = 0.0007
buyAmount = 0
bigBuy = 0
mediumBuy = 0
lowBuy = 0
totalEntry = 0
totalInvest = 0
btcWallet = 0

for index, row in dfTest.iterrows():
    if row['close'] <= 0.5 * row['LAST_ATH']:
        buyAmount = 2 * weeklyAmount
        bigBuy+=1
    elif row['close'] > 0.5 * row['LAST_ATH'] and row['close'] <= 0.8 * row['LAST_ATH']:
        mediumBuy+=1
        buyAmount = 1 * weeklyAmount
    elif row['close'] > 0.8 * row['LAST_ATH']:
        lowBuy+=1
        buyAmount = 0.5 * weeklyAmount
        # buyAmount = 0
    totalInvest += buyAmount
    buyBTC = buyAmount / row['close']
    btcWallet += buyBTC - takerFee * buyBTC
    totalEntry += 1

resultInDollar = btcWallet * dfTest.iloc[-1]['close']
perfInPct = (resultInDollar - totalInvest)/totalInvest
buyAndHoldPerf = (dfTest.iloc[-1]['close'] - dfTest.iloc[0]['close'])/dfTest.iloc[0]['close']
print('Buy',bigBuy,'time',2*weeklyAmount,'$')   
print('Buy',mediumBuy,'time',1*weeklyAmount,'$')   
print('Buy',lowBuy,'time',0.5*weeklyAmount,'$')   
print('Total invest', totalInvest, '$')
print('Final wallet', round(btcWallet,3), 'BTC')
print('Final wallet equivalent', round(resultInDollar,2), '$')
print('Performance',round(perfInPct*100,2), '%')
print('Buy and Hold performance', round(buyAndHoldPerf*100,2), '%')

Buy 103 time 60 $
Buy 86 time 30 $
Buy 56 time 15.0 $
Total invest 9600.0 $
Final wallet 1.217 BTC
Final wallet equivalent 48291.92 $
Performance 403.04 %
Buy and Hold performance 871.02 %
